In [1]:
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
import torch
import os
import re
import warnings

/home/ts777/datascience/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
if torch.cuda.is_available():
    device = "cuda"
    current_device = torch.cuda.current_device()
    print(torch.cuda.get_device_name(current_device))
else:
    print("CPU")

NVIDIA GeForce RTX 3060


In [4]:
dataset = load_from_disk("datasets/imdb/")

In [32]:
# first_train = dataset["train"][:15_000]
# first_test = dataset["test"][:3_000]

In [6]:
first_test.keys()

dict_keys(['label', 'title', 'content'])

In [ ]:
"distilbert/distilbert-base-uncased"

In [5]:
model_path = "xlnet/xlnet-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, data_dict:dict):
        super(DataSet, self).__init__()
        self.text = data_dict["content"]
        self.label = data_dict["label"]
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        encoded_items = tokenizer(self.text[idx],
                                  truncation=True,
                                  max_length=512,
                                  padding="max_length",
                                  return_tensors="pt")
        return {
            "input_ids": encoded_items["input_ids"].flatten(),
            "attention_mask": encoded_items["attention_mask"].flatten(),
            "label": torch.tensor([self.label[idx]])
        }

def get_dataset(data_dict):
    return Dataset(data_dict)

In [6]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    y_true = labels
    y_pred = np.argmax(predictions, axis=1)
    scores = predictions[:,1]

    accuracy_score_ = accuracy_score(y_true, y_pred)
    
    precision_score_0 = precision_score(y_true, y_pred, pos_label=False)
    precision_score_1 = precision_score(y_true, y_pred, pos_label=True)
    
    recall_score_0 = recall_score(y_true, y_pred, pos_label=False)
    recall_score_1 = recall_score(y_true, y_pred, pos_label=True)

    f1_score_0 = f1_score(y_true, y_pred, pos_label=False)
    f1_score_1 = f1_score(y_true, y_pred, pos_label=True)

    roc_auc = roc_auc_score(y_true, scores)

    result = {
        "Accuracy": accuracy_score_,
        "Precision-0": precision_score_0,
        "Recall-0": recall_score_0,
        "F1-0": f1_score_0,
        "Precision-1": precision_score_1,
        "Recall-1": recall_score_1,
        "F1-1": f1_score_1, 
        "ROC-AUC": roc_auc
    }
    
    return {k: round(v*100, 4) for k, v in result.items()}

In [8]:
# train_dataset = DataSet(first_train)
# test_dataset = DataSet(first_test)

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

In [8]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:18<00:00, 1318.32 examples/s]


In [9]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=512)

In [10]:
training_arguments = TrainingArguments(
    output_dir="tmp_models/impb_xlnet_finetuned",
    logging_dir="tmp_models/impb_xlnet_finetuned",
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    num_train_epochs=3,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=.01,
    dataloader_num_workers=8,
    save_total_limit=6, 
    lr_scheduler_type="linear",
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"], 
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [46]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss,Validation Loss,Accuracy,Precision-0,Recall-0,F1-0,Precision-1,Recall-1,F1-1,Roc-auc
1000,0.583100,0.550936,79.788000,77.429100,84.088000,80.621300,82.590800,75.488000,78.879800,87.516400
2000,0.471300,0.420096,82.080000,83.113100,80.520000,81.796000,81.109400,83.640000,82.355300,89.779700
3000,0.400800,0.436713,84.220000,87.637500,79.680000,83.469500,81.371500,88.760000,84.905300,90.091500
4000,0.349600,0.491053,85.504000,84.312700,87.240000,85.751400,86.781000,83.768000,85.247900,92.539700
5000,0.322900,0.426324,85.344000,82.309500,90.040000,86.001400,89.007600,80.648000,84.621800,93.109800
6000,0.337000,0.412179,86.236000,84.353400,88.976000,86.603100,88.336900,83.496000,85.848200,93.672100
7000,0.274900,0.437403,86.452000,87.162500,85.496000,86.321200,85.768100,87.408000,86.580300,93.709100
8000,0.277900,0.474495,86.664000,86.189200,87.320000,86.750900,87.151400,86.008000,86.575900,93.779200
9000,0.269100,0.436916,86.620000,86.097300,87.344000,86.716200,87.158000,85.896000,86.522400,93.972300


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=9375, training_loss=0.3617043375651042, metrics={'train_runtime': 3955.5112, 'train_samples_per_second': 18.961, 'train_steps_per_second': 2.37, 'total_flos': 9364642001374848.0, 'train_loss': 0.3617043375651042, 'epoch': 3.0})

In [43]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss,Validation Loss,Accuracy,Precision-0,Recall-0,F1-0,Precision-1,Recall-1,F1-1,Roc-auc
1000,0.497500,0.539568,79.624000,84.452900,72.616000,78.088400,75.982300,86.632000,80.958400,85.881500
2000,0.677600,0.694017,50.000000,50.000000,100.000000,66.666700,0.000000,0.000000,0.000000,85.306400
3000,0.694200,0.693250,50.000000,0.000000,0.000000,0.000000,50.000000,100.000000,66.666700,85.387700
4000,0.693500,0.693728,50.000000,50.000000,100.000000,66.666700,0.000000,0.000000,0.000000,84.743100
5000,0.693500,0.693177,50.000000,50.000000,100.000000,66.666700,0.000000,0.000000,0.000000,85.555700
6000,0.693200,0.693641,50.000000,0.000000,0.000000,0.000000,50.000000,100.000000,66.666700,85.718600
7000,0.691400,0.693420,50.000000,0.000000,0.000000,0.000000,50.000000,100.000000,66.666700,55.201800
8000,0.694000,0.693238,50.000000,0.000000,0.000000,0.000000,50.000000,100.000000,66.666700,75.835200
9000,0.693300,0.693158,50.000000,0.000000,0.000000,0.000000,50.000000,100.000000,66.666700,23.411700


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=9375, training_loss=0.670715439453125, metrics={'train_runtime': 3964.7303, 'train_samples_per_second': 18.917, 'train_steps_per_second': 2.365, 'total_flos': 9364642001374848.0, 'train_loss': 0.670715439453125, 'epoch': 3.0})

In [13]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision-0,Recall-0,F1-0,Precision-1,Recall-1,F1-1,Roc-auc
1000,0.618500,0.692600,52.100000,0.000000,0.000000,0.000000,52.100000,100.000000,68.507600,63.993100
2000,0.717700,0.710614,52.100000,0.000000,0.000000,0.000000,52.100000,100.000000,68.507600,35.546900
3000,0.711400,0.693444,47.900000,47.900000,100.000000,64.773500,0.000000,0.000000,0.000000,47.430600
4000,0.711100,0.706112,47.900000,47.900000,100.000000,64.773500,0.000000,0.000000,0.000000,34.380400
5000,0.703300,0.695088,47.900000,47.900000,100.000000,64.773500,0.000000,0.000000,0.000000,69.974700


/home/ts777/datascience/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ts777/datascience/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ts777/datascience/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ts777/datasci

TrainOutput(global_step=5625, training_loss=0.6929692654079861, metrics={'train_runtime': 6261.5589, 'train_samples_per_second': 7.187, 'train_steps_per_second': 0.898, 'total_flos': 1.281961257984e+16, 'train_loss': 0.6929692654079861, 'epoch': 3.0})

In [17]:
trainer.train()

Step,Training Loss,Validation Loss


[[0.05322411 0.3455544 ]
 [0.05331015 0.34534007]
 [0.05341507 0.34531856]
 ...
 [0.05275582 0.34620592]
 [0.05296462 0.34543565]
 [0.05318213 0.34570983]] [1 1 0 ... 0 1 1]


TypeError: argument 'ids': 'float' object cannot be interpreted as an integer